# Notebook 2: Load Model and Make Predictions
## SWaT Dataset - SCADA Anomaly Detection

**Purpose:** Load the pre-trained model and make predictions on test data

**Prerequisites:** Must have run `01_Train_IsolationForest_SWaT.ipynb` first!

**Run this notebook:**
- To test on attack dataset
- Daily for new data
- Whenever you need predictions (fast - no retraining!)

---

## Step 1: Mount Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
print("Google Drive mounted successfully!")

## Step 2: Configuration

In [ ]:
# =============================================================
# CONFIGURATION
# =============================================================

# Path to your SWaT TEST data (contains attacks)
TEST_DATA_PATH = '/content/drive/MyDrive/4th7thsemproject/SWaT_Attack.csv'

# Paths to saved model artifacts (must match training notebook!)
MODEL_PATH = '/content/drive/MyDrive/4th7thsemproject/models/isolation_forest_model.pkl'
SCALER_PATH = '/content/drive/MyDrive/4th7thsemproject/models/scaler.pkl'
FEATURES_PATH = '/content/drive/MyDrive/4th7thsemproject/models/feature_columns.pkl'

# Path to save prediction results
RESULTS_SAVE_PATH = '/content/drive/MyDrive/4th7thsemproject/results/predictions.csv'

# =============================================================
# LABEL CONFIGURATION (for evaluation)
# =============================================================

# How labels are encoded in your test data
NORMAL_LABEL = 'Normal'
ATTACK_LABEL = 'Attack'

print("Configuration loaded!")
print(f"Test data: {TEST_DATA_PATH}")
print(f"Model: {MODEL_PATH}")

## Step 3: Import Libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import (
    confusion_matrix, 
    classification_report, 
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    roc_auc_score,
    roc_curve
)
import joblib
import os
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

print("All libraries imported successfully!")

## Step 4: Load Saved Model, Scaler, and Features

In [ ]:
# Verify files exist before loading
print("Checking for saved model artifacts...\n")

files_to_check = [
    ("Model", MODEL_PATH),
    ("Scaler", SCALER_PATH),
    ("Features", FEATURES_PATH)
]

all_files_exist = True
for name, path in files_to_check:
    if os.path.exists(path):
        print(f"✅ {name}: Found")
    else:
        print(f"❌ {name}: NOT FOUND - {path}")
        all_files_exist = False

if not all_files_exist:
    raise FileNotFoundError("\n⚠️ Missing model files! Run '01_Train_IsolationForest_SWaT.ipynb' first!")

print("\n✅ All model files found!")

In [ ]:
# Load the trained model
print("Loading model...")
clf = joblib.load(MODEL_PATH)
print(f"✅ Model loaded: {type(clf).__name__}")
print(f"   - n_estimators: {clf.n_estimators}")
print(f"   - contamination: {clf.contamination}")

# Load the scaler
print("\nLoading scaler...")
scaler = joblib.load(SCALER_PATH)
print(f"✅ Scaler loaded: {type(scaler).__name__}")

# Load feature column names
print("\nLoading feature columns...")
feature_columns = joblib.load(FEATURES_PATH)
print(f"✅ Feature columns loaded: {len(feature_columns)} features")

## Step 5: Load Test Data

In [ ]:
# Load the test data
print(f"Loading test data from: {TEST_DATA_PATH}")
print("This may take a moment for large files...")

test_df = pd.read_csv(TEST_DATA_PATH)

print(f"\n✅ Test data loaded successfully!")
print(f"   Shape: {test_df.shape[0]:,} rows × {test_df.shape[1]} columns")

In [ ]:
# Preview test data
print("First 5 rows:")
display(test_df.head())

print("\nColumn names:")
print(test_df.columns.tolist())

In [ ]:
# Find and extract labels (ground truth)
label_col = None
for col in ['Normal/Attack', ' Normal/Attack', 'label', 'Label', 'Attack', 'attack']:
    if col in test_df.columns:
        label_col = col
        break

if label_col:
    print(f"Found label column: '{label_col}'")
    print(f"\nValue counts:")
    print(test_df[label_col].value_counts())
    
    # Store ground truth
    y_true_raw = test_df[label_col].values
    
    # Convert to binary (1 = Normal, -1 = Anomaly) to match Isolation Forest output
    y_true = np.where(
        (y_true_raw == NORMAL_LABEL) | (y_true_raw == 'Normal') | (y_true_raw == 0),
        1,   # Normal
        -1   # Anomaly/Attack
    )
    
    print(f"\nConverted labels:")
    print(f"  Normal (1):  {(y_true == 1).sum():,}")
    print(f"  Anomaly (-1): {(y_true == -1).sum():,}")
else:
    print("⚠️ No label column found - will only show predictions without evaluation")
    y_true = None

## Step 6: Prepare Test Features

In [ ]:
# Check that all expected features are present
missing_features = [f for f in feature_columns if f not in test_df.columns]
if missing_features:
    print(f"⚠️ Missing features in test data: {missing_features}")
    raise ValueError("Test data is missing required features!")
else:
    print(f"✅ All {len(feature_columns)} features found in test data")

In [ ]:
# Extract feature matrix (using same columns as training!)
X_test = test_df[feature_columns].values

print(f"Test feature matrix shape: {X_test.shape}")
print(f"  - {X_test.shape[0]:,} samples")
print(f"  - {X_test.shape[1]} features")

In [ ]:
# Handle missing values (same as training)
missing = np.isnan(X_test).sum()
if missing > 0:
    print(f"⚠️ Found {missing} missing values")
    print("Filling with training means (from scaler)...")
    nan_indices = np.where(np.isnan(X_test))
    X_test[nan_indices] = np.take(scaler.mean_, nan_indices[1])
    print("✅ Missing values handled")
else:
    print("✅ No missing values found")

In [ ]:
# Scale test data using the SAVED scaler (important!)
print("Scaling test data with saved scaler...")
X_test_scaled = scaler.transform(X_test)  # Use transform, NOT fit_transform!
print("✅ Test data scaled")

## Step 7: Make Predictions

In [ ]:
# Make predictions
print("🔮 Making predictions...")
start_time = datetime.now()

y_pred = clf.predict(X_test_scaled)

end_time = datetime.now()
prediction_time = (end_time - start_time).total_seconds()

print(f"✅ Predictions complete!")
print(f"   Time: {prediction_time:.2f} seconds")
print(f"   Speed: {len(y_pred)/prediction_time:,.0f} predictions/second")

In [ ]:
# Get anomaly scores (more detailed than binary prediction)
anomaly_scores = clf.decision_function(X_test_scaled)

print("Prediction summary:")
print(f"  Normal (1):   {(y_pred == 1).sum():,} ({(y_pred == 1).sum()/len(y_pred)*100:.2f}%)")
print(f"  Anomaly (-1): {(y_pred == -1).sum():,} ({(y_pred == -1).sum()/len(y_pred)*100:.2f}%)")
print(f"\nAnomaly scores:")
print(f"  Min: {anomaly_scores.min():.4f}")
print(f"  Max: {anomaly_scores.max():.4f}")
print(f"  Mean: {anomaly_scores.mean():.4f}")

## Step 8: Evaluate Performance (if labels available)

In [ ]:
if y_true is not None:
    print("="*60)
    print("MODEL PERFORMANCE EVALUATION")
    print("="*60)
    
    # Calculate metrics
    accuracy = accuracy_score(y_true, y_pred)
    
    # For precision, recall, F1 - we care about detecting anomalies (label = -1)
    precision = precision_score(y_true, y_pred, pos_label=-1)
    recall = recall_score(y_true, y_pred, pos_label=-1)
    f1 = f1_score(y_true, y_pred, pos_label=-1)
    
    print(f"\n📊 Overall Metrics:")
    print(f"   Accuracy:  {accuracy:.4f} ({accuracy*100:.2f}%)")
    print(f"   Precision: {precision:.4f} (of predicted anomalies, how many were real)")
    print(f"   Recall:    {recall:.4f} (of real anomalies, how many did we catch)")
    print(f"   F1 Score:  {f1:.4f} (balance of precision and recall)")
else:
    print("⚠️ No ground truth labels - skipping evaluation")

In [ ]:
if y_true is not None:
    # Confusion Matrix
    print("\n📊 Confusion Matrix:")
    cm = confusion_matrix(y_true, y_pred, labels=[1, -1])
    
    print(f"\n                  Predicted")
    print(f"                  Normal  Anomaly")
    print(f"Actual Normal    {cm[0,0]:7,}  {cm[0,1]:7,}")
    print(f"Actual Anomaly   {cm[1,0]:7,}  {cm[1,1]:7,}")
    
    print(f"\nInterpretation:")
    print(f"  True Negatives (correctly identified normal):  {cm[0,0]:,}")
    print(f"  False Positives (false alarms):                {cm[0,1]:,}")
    print(f"  False Negatives (missed attacks):              {cm[1,0]:,}")
    print(f"  True Positives (correctly detected attacks):   {cm[1,1]:,}")

In [ ]:
if y_true is not None:
    # Detailed classification report
    print("\n📊 Detailed Classification Report:")
    print(classification_report(y_true, y_pred, target_names=['Normal', 'Anomaly'], labels=[1, -1]))

## Step 9: Visualizations

In [ ]:
# Confusion Matrix Heatmap
if y_true is not None:
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # Plot 1: Confusion Matrix
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                xticklabels=['Normal', 'Anomaly'],
                yticklabels=['Normal', 'Anomaly'],
                ax=axes[0])
    axes[0].set_xlabel('Predicted')
    axes[0].set_ylabel('Actual')
    axes[0].set_title('Confusion Matrix')
    
    # Plot 2: Anomaly Score Distribution
    scores_normal = anomaly_scores[y_true == 1]
    scores_anomaly = anomaly_scores[y_true == -1]
    
    axes[1].hist(scores_normal, bins=50, alpha=0.7, label='Normal', color='blue')
    axes[1].hist(scores_anomaly, bins=50, alpha=0.7, label='Anomaly', color='red')
    axes[1].axvline(x=0, color='black', linestyle='--', label='Decision Boundary')
    axes[1].set_xlabel('Anomaly Score')
    axes[1].set_ylabel('Frequency')
    axes[1].set_title('Anomaly Score Distribution')
    axes[1].legend()
    
    plt.tight_layout()
    plt.show()
else:
    plt.figure(figsize=(10, 5))
    plt.hist(anomaly_scores, bins=50, alpha=0.7, color='blue')
    plt.axvline(x=0, color='red', linestyle='--', label='Decision Boundary')
    plt.xlabel('Anomaly Score')
    plt.ylabel('Frequency')
    plt.title('Anomaly Score Distribution')
    plt.legend()
    plt.show()

In [ ]:
# ROC Curve (if labels available)
if y_true is not None:
    # Convert labels for ROC (1 = anomaly, 0 = normal)
    y_true_binary = (y_true == -1).astype(int)
    scores_for_roc = -anomaly_scores
    
    fpr, tpr, thresholds = roc_curve(y_true_binary, scores_for_roc)
    auc_score = roc_auc_score(y_true_binary, scores_for_roc)
    
    plt.figure(figsize=(8, 6))
    plt.plot(fpr, tpr, color='blue', lw=2, label=f'ROC Curve (AUC = {auc_score:.4f})')
    plt.plot([0, 1], [0, 1], color='gray', linestyle='--', label='Random Classifier')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate (Recall)')
    plt.title('ROC Curve - Anomaly Detection')
    plt.legend(loc='lower right')
    plt.grid(True, alpha=0.3)
    plt.show()
    
    print(f"\n📊 AUC Score: {auc_score:.4f}")
    print("   (1.0 = perfect, 0.5 = random, <0.5 = worse than random)")

## Step 10: Save Results

In [ ]:
# Create results directory if needed
results_dir = os.path.dirname(RESULTS_SAVE_PATH)
if results_dir and not os.path.exists(results_dir):
    os.makedirs(results_dir)
    print(f"Created directory: {results_dir}")

# Create results DataFrame
results_df = test_df.copy()
results_df['Prediction'] = np.where(y_pred == 1, 'Normal', 'Anomaly')
results_df['Anomaly_Score'] = anomaly_scores
results_df['Predicted_Label'] = y_pred

if y_true is not None:
    results_df['Correct'] = (y_pred == y_true)

# Save to CSV
print(f"\nSaving results to: {RESULTS_SAVE_PATH}")
results_df.to_csv(RESULTS_SAVE_PATH, index=False)
print("✅ Results saved!")

In [ ]:
# Preview saved results
print("\nPreview of saved results:")
display(results_df[['Prediction', 'Anomaly_Score', 'Predicted_Label']].head(10))

## Summary

In [ ]:
print("\n" + "="*60)
print("INFERENCE COMPLETE - SUMMARY")
print("="*60)
print(f"\n📊 Test Dataset:")
print(f"   - Samples: {X_test.shape[0]:,}")
print(f"   - Features: {X_test.shape[1]}")
print(f"\n🔮 Predictions:")
print(f"   - Normal:  {(y_pred == 1).sum():,}")
print(f"   - Anomaly: {(y_pred == -1).sum():,}")
print(f"   - Prediction time: {prediction_time:.2f} seconds")

if y_true is not None:
    print(f"\n📈 Performance:")
    print(f"   - Accuracy:  {accuracy:.4f}")
    print(f"   - Precision: {precision:.4f}")
    print(f"   - Recall:    {recall:.4f}")
    print(f"   - F1 Score:  {f1:.4f}")

print(f"\n💾 Results saved to: {RESULTS_SAVE_PATH}")
print("="*60)